# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234690626820                   -0.50    7.0         
  2   -7.250193914285       -1.81       -1.40    1.0   10.1ms
  3   -7.251187160231       -3.00       -2.05    2.0   11.0ms
  4   -7.251204078360       -4.77       -2.08    2.0   12.3ms
  5   -7.251317401701       -3.95       -2.49    1.0   10.2ms
  6   -7.251338103889       -4.68       -3.19    1.0   10.0ms
  7   -7.251338730898       -6.20       -3.66    2.0   12.2ms
  8   -7.251338785613       -7.26       -3.95    2.0   12.4ms
  9   -7.251338797979       -7.91       -4.68    1.0   10.6ms
 10   -7.251338798654       -9.17       -5.24    2.0   12.1ms
 11   -7.251338798690      -10.44       -5.49    2.0   12.2ms
 12   -7.251338798703      -10.88       -6.16    1.0   10.6ms
 13   -7.251338798704      -11.96       -6.38    3.0   14.4ms
 14   -7.251338798705      -12.78       -6.81    1.0   10.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07878325439177834
[ Info: Arnoldi iteration step 2: normres = 0.5534171532386455
[ Info: Arnoldi iteration step 3: normres = 0.8290316854549691
[ Info: Arnoldi iteration step 4: normres = 0.26765663678362467
[ Info: Arnoldi iteration step 5: normres = 0.4332474654355491
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.24e-02, 5.42e-02, 3.42e-01, 2.59e-01, 1.04e-02)
[ Info: Arnoldi iteration step 6: normres = 0.29759320890354996
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.56e-03, 7.18e-02, 2.60e-01, 9.13e-02, 7.92e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07688314094792136
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.04e-04, 6.11e-03, 1.67e-02, 2.82e-02, 5.85e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07825567633253022
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.85e